# Add Fossil Classification for a given Holding File
## Prepare environment

In [1]:
import os
os.chdir('..')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from fossil_classification import *
from enrich_holdings import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = "{:,.2f}".format

# Classify quarterly holdings file

In [4]:
# TODO: add better matching for non-Israeli companies, using Figi as name normalization, fall back to fuzzy matching on normalized name and then fuzzy matching on original name
# TODO: fix is_fossil_conflict - missing cases at the moment
# TODO: FFF false positives - short names? other cases?

In [5]:
company_or_fund_level = "company"
year = "2023"
q = "4"
folder_path = "data/downloaded reports/" + company_or_fund_level + " reports/" + year + "Q" + q + "/"
holdings_path = folder_path + "holdings_for_cls.csv"

In [ ]:
classify_holdings(holdings_path)

# Manual Review
In a google spreadsheet or excel:
https://drive.google.com/drive/u/1/folders/1Sik1gfWt6Ps37aCWxXukjKMqhG8N5FCV

Download the fully classified file into a csv, then use it in holding_cls_path to update prev_class (see below).

## Tips
1. Look at the output of the script, review conflicting classification (by ISIN, LEI, Israeli security number)
2. Look at holdings that get is_fossil_conflict=True
 <br>**TODO: fix is_fossil_conflict function**
3. Sort by security name, Israeli security number or ISIN for faster manual classification
4. Carefully review holdings that have only is_fossil by FFF name match, as there are false matches.
<br>Review both holdings for which all the other is_fossil_x flags are null, and such that have is_fossil by FFF = 1and other is_fossil_x = 0
5. Review holdings from suspicious industries: energy, oil and gas, utilities, materials.

## False Positives
* URI
* NOVO-NORDISK
* Global-E

# Add classification results to prev_class

In [6]:
holdings_cls_path = folder_path + "holdings_for_cls with fossil classification - reviewed.csv"
prev_class_path = "data_sources/prev_class.csv"
update_prev_class(holdings_cls_path, prev_class_path)
# prev_class_fixed = add_all_id_types_to_holdings(prev_class, tlv_s2i, isin2lei)

Adding classifications to prev_class, saving the previous version as data_sources/prev_class backup/prev_class 2024-06-10 15-21-57.csv


# Classify fund holdings
## Israeli funds
Data is scraped from https://mayaapi.tase.co.il/api/fund/details?fundId=
<br>Page address: https://maya.tase.co.il/fund/5132287?view=assets

In [ ]:
import json

In [ ]:
response_directory = "data/holdings_for_classification/5132287/"
response_path = response_directory + "response.json"
fund = pd.read_json(response_path, orient="index")
assets = pd.DataFrame(fund.loc["AssetCompostion"][0]['Assets'])
# holdings["AssetCompostion"].head()
cols_rename = {
    'AssetName': 'שם המנפיק/שם נייר ערך',
    'IdentityCd': 'מספר ני"ע',
    'Id': 'fund_id'
}
assets = assets.rename(cols_rename, axis=1)
assets["מספר מנפיק"] = '00'
assets["מספר תאגיד"] = '00'
assets.to_csv(response_directory+"assets.csv", index=False)

In [ ]:
classify_holdings(response_directory+"assets.csv")

## holdings CSV file

In [ ]:
holdings_csv_dir = "/Users/urimarom/PycharmProjects/fossil_classification/data/holdings_for_classification/IE000PSF3A70/"
holdings_filename = 'fund_weights.csv'
holdings_csv_path = holdings_csv_dir + holdings_filename
holdings = pd.read_csv(holdings_csv_path)
cols_rename = {
    'Name': 'שם המנפיק/שם נייר ערך',
    'ISIN': 'מספר ני"ע',
    'Type of Security': 'holding_type'
}
holdings = holdings.rename(cols_rename, axis=1)
# fix missing columns
holdings["מספר מנפיק"] = '00'
holdings["מספר תאגיד"] = '00'
holdings["is_fossil_prev_il_sec_num"] = np.nan
holdings.to_csv(holdings_csv_dir+"fund_weights_fixed.csv", index=False)

In [ ]:
classify_holdings(holdings_csv_dir+"fund_weights_fixed.csv")